In [1]:
import pandas as pd
import os
from PIL import Image
from keras.utils import img_to_array, load_img
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

df = pd.DataFrame(columns=['imagePath', 'label'])

directories = {
    "ayam_goreng": "./dataset/ayam_goreng/",
    "ayam_pop": "./dataset/ayam_pop/",
    "daging_rendang": "./dataset/daging_rendang/",
    "dendeng_batokok": "./dataset/dendeng_batokok/",
    "gulai_ikan": "./dataset/gulai_ikan/",
    "gulai_tambusu": "./dataset/gulai_tambusu/",
    "telur_balado": "./dataset/telur_balado/",
    "telur_dadar": "./dataset/telur_dadar/",
    "tahu": "./dataset/tahu/",
    "daun_singkong": "./dataset/daun_singkong/",
    "nangka": "./dataset/nangka/",
    "perkedel": "./dataset/perkedel/",
    "nasi" : "./dataset/nasi/"
}

for label, directory in directories.items():
    for i in os.listdir(directory):
        df = pd.concat([df, pd.DataFrame({'imagePath': [f"{directory}/{i}"], 'label': [label]})])

df = df.reset_index(drop=True)

imgs = []

for image_path in df['imagePath']:
    img = load_img(image_path, target_size=(127, 127)).convert('RGB')
    img_array = img_to_array(img) / 128
    imgs.append(img_array)

df['img'] = imgs

c:\Users\lenovo\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model2 = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(127, 127, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(directories), activation='softmax')
])

model2.compile(optimizer=Adam(learning_rate=01.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

def cutout(image, mask_size=20, mask_value=0):
    h, w, _ = image.shape
    top = np.random.randint(0 - mask_size // 2, h - mask_size)
    left = np.random.randint(0 - mask_size // 2, w - mask_size)
    bottom = top + mask_size
    right = left + mask_size
    image[max(0, top):min(h, bottom), max(0, left):min(w, right), :] = mask_value
    return image

# Define a new preprocessing function that applies the cutout technique
def augment_image(image):
    image = image + np.random.normal(0, 0.05, image.shape)  # Gaussian noise
    image = cutout(image)  # Cutout
    return image

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.25,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
     preprocessing_function=augment_image
)
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
x = np.array(df['img'].tolist())
y = le.fit_transform(df['label'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
datagen.fit(x_train)

In [4]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model2.fit(datagen.flow(x_train, y_train, batch_size=64),
                    validation_data=(x_test, y_test),
                    epochs=50,
                    callbacks=[early_stopping])

loss, accuracy = model2.evaluate(x_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/50
34/34 [==============================] - 22s 311ms/step - loss: 819334.6875 - accuracy: 0.0866 - val_loss: 59025633280.0000 - val_accuracy: 0.0654
Epoch 2/50
34/34 [==============================] - 7s 203ms/step - loss: 2.6589 - accuracy: 0.0702 - val_loss: 7289560064.0000 - val_accuracy: 0.0617
Epoch 3/50
34/34 [==============================] - 7s 193ms/step - loss: 2.6501 - accuracy: 0.0824 - val_loss: 316901664.0000 - val_accuracy: 0.1121
Epoch 4/50
34/34 [==============================] - 6s 186ms/step - loss: 23211.7754 - accuracy: 0.0786 - val_loss: 96314048512.0000 - val_accuracy: 0.1084
Epoch 5/50
34/34 [==============================] - 6s 184ms/step - loss: 2.6366 - accuracy: 0.0847 - val_loss: 7826061312.0000 - val_accuracy: 0.0804
Epoch 6/50
34/34 [==============================] - 7s 189ms/step - loss: 2.6335 - accuracy: 0.0744 - val_loss: 513642304.0000 - val_accuracy: 0.0636
Epoch 7/50
34/34 [==============================] - 6s 183ms/step - loss: 2.6235 - a